In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        # implementing other type of resnet where instead of adding original layer , adding the convoluted layer of kernel 1
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        # Inheriting from the same parent node
        super(ResNet, self).__init__()
        self.in_planes = 64
        # 1. kernel 7 by 7 has to be used then max pooling, instead used 3 by 3 to get simialr output
        # self.conv1 = nn.Conv2d(input channels,output channels , kernel_size,stride, padding, bias=False)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
        # batch normalization
        self.bn1 = nn.BatchNorm2d(64)
        # then calling the sequential layers ( Basicblock, 64, 2 , 1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        # builds 2 layers everytime as strides are 2 everytime to build resnet
        for stride in strides:
            # basicblock() 
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        # setting the dimensions by reshaping it , just cvonverting 4d into 2d
        out = self.linear(out)
        return out


def ResNet18():
    # 1. Resnet in_it runs and 
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [4]:
net = ResNet18()

In [5]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,  momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for i in range(epoch):
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [6]:
train(5)


Epoch: 5
Loss: 2.419 | Acc: 5.469% (7/128)
Loss: 2.354 | Acc: 8.594% (22/256)
Loss: 2.326 | Acc: 9.375% (36/384)
Loss: 2.309 | Acc: 10.742% (55/512)
Loss: 2.294 | Acc: 12.031% (77/640)
Loss: 2.274 | Acc: 13.151% (101/768)
Loss: 2.267 | Acc: 14.062% (126/896)
Loss: 2.244 | Acc: 15.039% (154/1024)
Loss: 2.212 | Acc: 16.146% (186/1152)
Loss: 2.186 | Acc: 17.188% (220/1280)
Loss: 2.176 | Acc: 17.543% (247/1408)
Loss: 2.158 | Acc: 18.620% (286/1536)
Loss: 2.147 | Acc: 19.050% (317/1664)
Loss: 2.143 | Acc: 19.364% (347/1792)
Loss: 2.118 | Acc: 20.312% (390/1920)
Loss: 2.112 | Acc: 20.508% (420/2048)
Loss: 2.096 | Acc: 21.140% (460/2176)
Loss: 2.080 | Acc: 21.311% (491/2304)
Loss: 2.078 | Acc: 21.669% (527/2432)
Loss: 2.071 | Acc: 22.148% (567/2560)
Loss: 2.058 | Acc: 22.768% (612/2688)
Loss: 2.051 | Acc: 22.905% (645/2816)
Loss: 2.046 | Acc: 23.030% (678/2944)
Loss: 2.035 | Acc: 23.405% (719/3072)
Loss: 2.024 | Acc: 23.875% (764/3200)
Loss: 2.020 | Acc: 24.189% (805/3328)
Loss: 2.011 | Acc:

Loss: 1.486 | Acc: 45.598% (12140/26624)
Loss: 1.486 | Acc: 45.600% (12199/26752)
Loss: 1.484 | Acc: 45.655% (12272/26880)
Loss: 1.483 | Acc: 45.705% (12344/27008)
Loss: 1.481 | Acc: 45.751% (12415/27136)
Loss: 1.480 | Acc: 45.811% (12490/27264)
Loss: 1.479 | Acc: 45.853% (12560/27392)
Loss: 1.477 | Acc: 45.941% (12643/27520)
Loss: 1.475 | Acc: 46.018% (12723/27648)
Loss: 1.473 | Acc: 46.051% (12791/27776)
Loss: 1.472 | Acc: 46.112% (12867/27904)
Loss: 1.471 | Acc: 46.144% (12935/28032)
Loss: 1.470 | Acc: 46.183% (13005/28160)
Loss: 1.469 | Acc: 46.225% (13076/28288)
Loss: 1.467 | Acc: 46.294% (13155/28416)
Loss: 1.466 | Acc: 46.325% (13223/28544)
Loss: 1.464 | Acc: 46.376% (13297/28672)
Loss: 1.463 | Acc: 46.431% (13372/28800)
Loss: 1.461 | Acc: 46.498% (13451/28928)
Loss: 1.460 | Acc: 46.551% (13526/29056)
Loss: 1.458 | Acc: 46.597% (13599/29184)
Loss: 1.457 | Acc: 46.633% (13669/29312)
Loss: 1.456 | Acc: 46.671% (13740/29440)
Loss: 1.454 | Acc: 46.743% (13821/29568)
Loss: 1.452 | Ac

Loss: 30.149 | Acc: 54.353% (28359/52176)
Loss: 28.525 | Acc: 54.397% (28452/52304)
Loss: 27.062 | Acc: 54.444% (28546/52432)
Loss: 25.751 | Acc: 54.492% (28641/52560)
Loss: 24.571 | Acc: 54.521% (28726/52688)
Loss: 23.490 | Acc: 54.567% (28820/52816)
Loss: 22.513 | Acc: 54.584% (28899/52944)
Loss: 21.611 | Acc: 54.607% (28981/53072)
Loss: 20.783 | Acc: 54.641% (29069/53200)
Loss: 20.012 | Acc: 54.686% (29163/53328)
Loss: 19.301 | Acc: 54.725% (29254/53456)
Loss: 18.645 | Acc: 54.761% (29343/53584)
Loss: 18.037 | Acc: 54.781% (29424/53712)
Loss: 17.466 | Acc: 54.807% (29508/53840)
Loss: 16.933 | Acc: 54.838% (29595/53968)
Loss: 16.431 | Acc: 54.882% (29689/54096)
Loss: 15.961 | Acc: 54.909% (29774/54224)
Loss: 15.518 | Acc: 54.940% (29861/54352)
Loss: 15.102 | Acc: 54.960% (29942/54480)
Loss: 14.710 | Acc: 54.979% (30023/54608)
Loss: 14.334 | Acc: 55.019% (30115/54736)
Loss: 13.977 | Acc: 55.065% (30211/54864)
Loss: 13.639 | Acc: 55.106% (30304/54992)
Loss: 13.319 | Acc: 55.149% (30398

Loss: 3.107 | Acc: 60.106% (46671/77648)
Loss: 3.097 | Acc: 60.133% (46769/77776)
Loss: 3.086 | Acc: 60.154% (46862/77904)
Loss: 3.076 | Acc: 60.174% (46955/78032)
Loss: 3.066 | Acc: 60.194% (47048/78160)
Loss: 3.055 | Acc: 60.219% (47144/78288)
Loss: 3.045 | Acc: 60.239% (47237/78416)
Loss: 3.035 | Acc: 60.262% (47332/78544)
Loss: 3.025 | Acc: 60.284% (47427/78672)
Loss: 3.015 | Acc: 60.301% (47517/78800)
Loss: 3.004 | Acc: 60.332% (47619/78928)
Loss: 2.994 | Acc: 60.362% (47720/79056)
Loss: 2.984 | Acc: 60.385% (47815/79184)
Loss: 2.974 | Acc: 60.404% (47908/79312)
Loss: 2.964 | Acc: 60.425% (48002/79440)
Loss: 2.954 | Acc: 60.459% (48106/79568)
Loss: 2.946 | Acc: 60.474% (48195/79696)
Loss: 2.936 | Acc: 60.499% (48293/79824)
Loss: 2.927 | Acc: 60.520% (48387/79952)
Loss: 2.918 | Acc: 60.532% (48474/80080)
Loss: 2.909 | Acc: 60.560% (48574/80208)
Loss: 2.900 | Acc: 60.572% (48661/80336)
Loss: 2.892 | Acc: 60.585% (48749/80464)
Loss: 2.883 | Acc: 60.603% (48841/80592)
Loss: 2.875 | Ac

Loss: 33.638 | Acc: 64.024% (65991/103072)
Loss: 32.316 | Acc: 64.042% (66091/103200)
Loss: 31.089 | Acc: 64.069% (66201/103328)
Loss: 29.955 | Acc: 64.093% (66308/103456)
Loss: 28.902 | Acc: 64.115% (66413/103584)
Loss: 27.920 | Acc: 64.141% (66522/103712)
Loss: 27.005 | Acc: 64.166% (66630/103840)
Loss: 26.149 | Acc: 64.193% (66740/103968)
Loss: 25.347 | Acc: 64.210% (66840/104096)
Loss: 24.596 | Acc: 64.230% (66943/104224)
Loss: 23.887 | Acc: 64.255% (67051/104352)
Loss: 23.221 | Acc: 64.270% (67149/104480)
Loss: 22.593 | Acc: 64.286% (67248/104608)
Loss: 21.998 | Acc: 64.305% (67350/104736)
Loss: 21.433 | Acc: 64.325% (67454/104864)
Loss: 20.897 | Acc: 64.348% (67560/104992)
Loss: 20.386 | Acc: 64.374% (67670/105120)
Loss: 19.903 | Acc: 64.395% (67774/105248)
Loss: 19.443 | Acc: 64.409% (67872/105376)
Loss: 19.002 | Acc: 64.430% (67976/105504)
Loss: 18.581 | Acc: 64.450% (68080/105632)
Loss: 18.179 | Acc: 64.474% (68188/105760)
Loss: 17.795 | Acc: 64.496% (68293/105888)
Loss: 17.42

...

Loss: 50.314 | Acc: 69.867% (106589/152560)
Loss: 47.933 | Acc: 69.885% (106706/152688)
Loss: 45.770 | Acc: 69.899% (106817/152816)
Loss: 43.795 | Acc: 69.913% (106928/152944)
Loss: 41.988 | Acc: 69.926% (107037/153072)
Loss: 40.320 | Acc: 69.942% (107151/153200)
Loss: 38.780 | Acc: 69.959% (107267/153328)
Loss: 37.352 | Acc: 69.976% (107382/153456)
Loss: 36.033 | Acc: 69.991% (107495/153584)
Loss: 34.799 | Acc: 70.009% (107612/153712)
Loss: 33.652 | Acc: 70.023% (107724/153840)
Loss: 32.577 | Acc: 70.037% (107834/153968)
Loss: 31.571 | Acc: 70.050% (107944/154096)
Loss: 30.623 | Acc: 70.068% (108062/154224)
Loss: 29.732 | Acc: 70.083% (108174/154352)
Loss: 28.893 | Acc: 70.099% (108289/154480)
Loss: 28.097 | Acc: 70.117% (108406/154608)
Loss: 27.348 | Acc: 70.129% (108515/154736)
Loss: 26.638 | Acc: 70.142% (108625/154864)
Loss: 25.963 | Acc: 70.155% (108735/154992)
Loss: 25.325 | Acc: 70.169% (108846/155120)
Loss: 24.718 | Acc: 70.182% (108956/155248)
Loss: 24.138 | Acc: 70.197% (109

Loss: 5.146 | Acc: 72.257% (127716/176752)
Loss: 5.123 | Acc: 72.268% (127828/176880)
Loss: 5.100 | Acc: 72.279% (127939/177008)
Loss: 5.078 | Acc: 72.287% (128046/177136)
Loss: 5.056 | Acc: 72.299% (128160/177264)
Loss: 5.034 | Acc: 72.305% (128264/177392)
Loss: 5.013 | Acc: 72.315% (128374/177520)
Loss: 4.991 | Acc: 72.324% (128482/177648)
Loss: 4.969 | Acc: 72.335% (128594/177776)
Loss: 4.948 | Acc: 72.349% (128711/177904)
Loss: 4.927 | Acc: 72.359% (128823/178032)
Loss: 4.906 | Acc: 72.370% (128934/178160)
Loss: 4.886 | Acc: 72.378% (129041/178288)
Loss: 4.866 | Acc: 72.388% (129151/178416)
Loss: 4.845 | Acc: 72.398% (129263/178544)
Loss: 4.825 | Acc: 72.409% (129374/178672)
Loss: 4.806 | Acc: 72.419% (129485/178800)
Loss: 4.786 | Acc: 72.427% (129593/178928)
Loss: 4.767 | Acc: 72.438% (129704/179056)
Loss: 4.747 | Acc: 72.452% (129823/179184)
Loss: 4.728 | Acc: 72.463% (129934/179312)
Loss: 4.709 | Acc: 72.474% (130047/179440)
Loss: 4.690 | Acc: 72.485% (130159/179568)
Loss: 4.672

Loss: 127.920 | Acc: 73.907% (148665/201152)
Loss: 115.147 | Acc: 73.919% (148784/201280)
Loss: 104.703 | Acc: 73.929% (148899/201408)
Loss: 95.997 | Acc: 73.941% (149017/201536)
Loss: 88.631 | Acc: 73.952% (149134/201664)
Loss: 82.318 | Acc: 73.962% (149250/201792)
Loss: 76.847 | Acc: 73.973% (149367/201920)
Loss: 72.060 | Acc: 73.983% (149482/202048)
Loss: 67.834 | Acc: 73.996% (149603/202176)
Loss: 64.078 | Acc: 74.008% (149722/202304)
Loss: 60.716 | Acc: 74.020% (149840/202432)
Loss: 57.691 | Acc: 74.031% (149957/202560)
Loss: 54.958 | Acc: 74.041% (150072/202688)
Loss: 52.471 | Acc: 74.051% (150187/202816)
Loss: 50.198 | Acc: 74.064% (150308/202944)
Loss: 48.119 | Acc: 74.074% (150423/203072)
Loss: 46.200 | Acc: 74.087% (150544/203200)
Loss: 44.431 | Acc: 74.098% (150662/203328)
Loss: 42.794 | Acc: 74.109% (150779/203456)
Loss: 41.276 | Acc: 74.119% (150894/203584)
Loss: 39.858 | Acc: 74.132% (151016/203712)
Loss: 38.536 | Acc: 74.143% (151133/203840)
Loss: 37.298 | Acc: 74.156% (

Loss: 6.062 | Acc: 75.835% (170793/225216)
Loss: 6.033 | Acc: 75.842% (170905/225344)
Loss: 6.004 | Acc: 75.852% (171026/225472)
Loss: 5.975 | Acc: 75.863% (171147/225600)
Loss: 5.947 | Acc: 75.871% (171262/225728)
Loss: 5.919 | Acc: 75.878% (171375/225856)
Loss: 5.891 | Acc: 75.885% (171489/225984)
Loss: 5.863 | Acc: 75.894% (171605/226112)
Loss: 5.836 | Acc: 75.901% (171719/226240)
Loss: 5.810 | Acc: 75.906% (171828/226368)
Loss: 5.783 | Acc: 75.913% (171941/226496)
Loss: 5.757 | Acc: 75.922% (172057/226624)
Loss: 5.731 | Acc: 75.929% (172171/226752)
Loss: 5.705 | Acc: 75.935% (172282/226880)
Loss: 5.680 | Acc: 75.942% (172395/227008)
Loss: 5.654 | Acc: 75.949% (172508/227136)
Loss: 5.629 | Acc: 75.957% (172624/227264)
Loss: 5.604 | Acc: 75.966% (172741/227392)
Loss: 5.580 | Acc: 75.971% (172849/227520)
Loss: 5.556 | Acc: 75.976% (172957/227648)
Loss: 5.531 | Acc: 75.986% (173078/227776)
Loss: 5.507 | Acc: 75.994% (173194/227904)
Loss: 5.483 | Acc: 76.001% (173307/228032)
Loss: 5.460

Loss: 3.221 | Acc: 77.192% (192721/249664)
Loss: 3.213 | Acc: 77.198% (192834/249792)
Loss: 3.206 | Acc: 77.203% (192946/249920)
Loss: 3.198 | Acc: 77.208% (193020/250000)


In [1]:
total = 0 
correct =0

with torch.no_grad():
    
    for data in testloader:
    
        inputs, labels=data

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

NameError: name 'torch' is not defined

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())
